# Backdoor AI - Mistral on Google Colab

This notebook helps you run **Mistral AI models** on Google Colab to use with your Backdoor AI application. Mistral models are known for their strong reasoning capabilities and good performance in a variety of tasks.

## How it works

1. This notebook will install Ollama on this Colab instance
2. You'll download the Mistral model
3. We'll set up Cloudflared to create a secure tunnel to your Ollama instance
4. You'll get a URL to use in your Backdoor AI settings

Let's get started!

## 1. Set up environment

First, let's install the required packages. We'll need Ollama and Cloudflared for tunneling.

In [ ]:
# Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Install cloudflared for tunneling
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

# Install other dependencies
!pip install -q requests pyngrok httpx ipywidgets

# Set up directories
!mkdir -p /tmp/ollama/models

## 2. Start Ollama server

Now we'll start the Ollama server in the background.

In [ ]:
import subprocess
import time
import requests
import json
from IPython.display import clear_output

# Start Ollama server in background
ollama_process = subprocess.Popen(
    ["ollama", "serve"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Wait for Ollama to start
print("Starting Ollama server...")
time.sleep(5)

# Check if Ollama is running
try:
    response = requests.get("http://localhost:11434/api/version")
    if response.status_code == 200:
        print(f"✅ Ollama is running! Version: {response.json().get('version')}")
    else:
        print(f"❌ Ollama returned unexpected status: {response.status_code}")
except Exception as e:
    print(f"❌ Failed to connect to Ollama: {e}")
    print("Trying to start again...")
    # Kill the previous process if it exists
    if ollama_process:
        ollama_process.terminate()
        time.sleep(2)
    # Try starting again
    !ollama serve &
    time.sleep(5)
    try:
        response = requests.get("http://localhost:11434/api/version")
        if response.status_code == 200:
            print(f"✅ Second attempt succeeded! Ollama is running. Version: {response.json().get('version')}")
    except:
        print("❌ Failed to start Ollama after multiple attempts.")

## 3. Download Mistral Model

Now, let's download the Mistral model for balanced performance.

In [ ]:
# Download Mistral
print("🚀 Downloading Mistral model...")
print("This model offers excellent reasoning and tool use capabilities.")
print("You'll see progress below. Please don't interrupt the process.")

# Run the download command
!ollama pull mistral:latest

# Verify the model is available
print("\n📋 Available models:")
!ollama list

## 4. Test the Mistral model

Let's make sure the model works by asking it something relevant to Backdoor AI usage.

In [ ]:
import requests
import json
from IPython.display import display, HTML

# Define a relevant prompt for Backdoor AI usage
test_prompt = """
I'm working on a Flask application and need help with structuring my API endpoints properly.
Can you explain how to effectively organize API routes for a RESTful service, with a specific 
example for a user management system with authentication?
"""

# Set up the API call
url = "http://localhost:11434/api/chat"
payload = {
    "model": "mistral:latest",
    "messages": [
        {"role": "user", "content": test_prompt}
    ],
    "stream": False
}

# Make the API call
try:
    print("Testing Mistral with a Flask development question...\n")
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        result = response.json()
        content = result.get("message", {}).get("content", "No content returned")
        print("✅ Model response:\n")
        print(content)
    else:
        print(f"❌ Error: Server returned status {response.status_code}")
        print(response.text)
except Exception as e:
    print(f"❌ Error testing model: {str(e)}")

## 5. Set up a tunnel to access your Ollama instance

Now we'll set up a Cloudflare tunnel so your Backdoor AI application can access this Ollama instance.

In [ ]:
import subprocess
import threading
import time
import re
from IPython.display import display, HTML
import ipywidgets as widgets

# Function to run cloudflared tunnel in a separate thread
def run_tunnel():
    global tunnel_process, tunnel_url
    tunnel_process = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", "http://localhost:11434"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )
    
    # Extract tunnel URL
    tunnel_url = None
    url_pattern = re.compile(r'https://[\w.-]+\.trycloudflare\.com')
    
    while True:
        line = tunnel_process.stdout.readline()
        if not line and tunnel_process.poll() is not None:
            break
        
        print(line.strip())
        match = url_pattern.search(line)
        if match and not tunnel_url:
            tunnel_url = match.group(0)
            tunnel_info.value = f"<div style='padding: 10px; background-color: #e6ffe6; border-radius: 5px;'><b>✅ Your Ollama API is accessible at:</b><br><code>{tunnel_url}</code><br><br>Use this URL in your Backdoor AI settings as the Ollama API Base URL.<br><br><b>Important:</b> In your Backdoor settings, make sure to select:<br>- Provider: Ollama<br>- Model: mistral:latest<br><br>Keep this notebook running while you're using Ollama with your app!</div>"

# Initialize global variables
tunnel_process = None
tunnel_url = None

# Create tunnel info widget
tunnel_info = widgets.HTML("<div style='padding: 10px; background-color: #fff3e6; border-radius: 5px;'>⏳ Creating secure tunnel to Ollama... (this may take a moment)</div>")
display(tunnel_info)

# Start tunnel
thread = threading.Thread(target=run_tunnel)
thread.daemon = True
thread.start()

# Wait for tunnel URL
attempts = 0
while attempts < 30 and not tunnel_url:
    time.sleep(1)
    attempts += 1
    
if not tunnel_url:
    tunnel_info.value = "<div style='padding: 10px; background-color: #ffe6e6; border-radius: 5px;'>❌ Failed to create tunnel. Check the output below for details.</div>"

## 6. Connect Backdoor AI to your Mistral instance

Once you have your tunnel URL, follow these steps to connect Backdoor AI to your Mistral instance:

1. Go to your Backdoor AI settings page
2. Select the "Ollama" tab
3. Select "Use Google Colab" as your setup method
4. In the "Ollama API URL" field, enter the tunnel URL from above
5. In the "Ollama Model" dropdown, select "mistral:latest"
6. Click "Save Settings"

### Benefits of Mistral for Backdoor AI

Mistral models offer several advantages for Backdoor AI application:
- Strong instruction following
- Good reasoning on complex problems
- Effective tool use capabilities
- Efficient context handling
- Strong knowledge of software development concepts

Mistral is particularly good for:
- Debugging and troubleshooting code
- Explaining complex programming concepts
- Working with Flask and web development tasks
- Analyzing and refactoring code

**Important Notes:**
- Keep this notebook running as long as you're using Ollama with your app
- The tunnel URL will change each time you restart this notebook
- Google Colab sessions have limited runtime (a few hours for free tier)
- Your model downloads will be lost when the Colab session ends

## 7. Keep the Colab session alive

Run the cell below to prevent Colab from disconnecting due to inactivity. This will create a small animation that keeps the session active.

In [ ]:
import time
import IPython.display
from IPython.display import HTML, display
import threading

def keep_alive():
    while True:
        time.sleep(60)  # Update every minute
        IPython.display.clear_output(wait=True)
        display(HTML(f'''
        <div style="padding: 10px; background-color: #f0f9ff; border-radius: 5px; width: 100%;">
            <h3>📡 Ollama Server Status</h3>
            <p><b>Tunnel URL:</b> {tunnel_url or "Not available"}</p>
            <p><b>Model:</b> mistral:latest</p>
            <p><b>Session active for:</b> {int(time.time() - start_time)} seconds</p>
            <p><b>Status:</b> Running</p>
            <div style="margin-top:10px; text-align:center;">
                <div class="spinner" style="display: inline-block; width: 20px; height: 20px; border: 3px solid rgba(0,0,0,.3); border-radius: 50%; border-top-color: #3498db; animation: spin 1s ease-in-out infinite;"></div>
                <style>
                    @keyframes spin { to { transform: rotate(360deg); } }
                </style>
            </div>
        </div>
        '''))

# Record start time
start_time = time.time()

# Start the keep-alive thread
keep_alive_thread = threading.Thread(target=keep_alive)
keep_alive_thread.daemon = True
keep_alive_thread.start()

# Initial display
display(HTML(f'''
<div style="padding: 10px; background-color: #f0f9ff; border-radius: 5px; width: 100%;">
    <h3>📡 Ollama Server Status</h3>
    <p><b>Tunnel URL:</b> {tunnel_url or "Not available"}</p>
    <p><b>Model:</b> mistral:latest</p>
    <p><b>Session active for:</b> 0 seconds</p>
    <p><b>Status:</b> Running</p>
    <div style="margin-top:10px; text-align:center;">
        <div class="spinner" style="display: inline-block; width: 20px; height: 20px; border: 3px solid rgba(0,0,0,.3); border-radius: 50%; border-top-color: #3498db; animation: spin 1s ease-in-out infinite;"></div>
        <style>
            @keyframes spin { to { transform: rotate(360deg); } }
        </style>
    </div>
</div>
'''))